# Tidy Data  by  Hadley Wickham
### Journal of Statistical Software, August 2014, Volume 59, Issue 10
### https://www.jstatsoft.org/article/view/v059i10

# 2. Defining tidy data

## 2.1. Data structure (physical layout)

- Most statistical datasets are rectangular **tables** made up of **rows** and **columns**.

|              | treatmenta | treatmentb |
| ------------ | ---------: | ---------: |
| John Smith   |        NaN |          2 |
| Jane Doe     |         16 |         11 |
| Mary Johnson |          3 |          1 |


## 2.2. Data semantics (meaning)

- A **dataset** is a collection of **values**, usually either numbers (if quantitative) or strings (if
qualitative).
- Every **value** belongs to a **variable (feature)** and an **observation (sample)**.
- A **variable** contains all values that measure the same underlying attribute (like height, temperature, duration) across units.
- An **observation** contains all values measured on the same unit (like a person, or a day, or a race) across attributes.


## 2.3. Tidy data

**Tidy data** is a standard way of mapping the meaning of a dataset to its structure. A dataset is
messy or tidy depending on how rows, columns and tables are matched up with observations,
variables and types. In tidy data:

1. Each **variable** forms a **column**.
2. Each **observation** forms a **row**.
3. Each **type** of observational unit forms a **table**.

| name         | trt  | result |
| ------------ | :--- | -----: |
| John Smith   | a    |    NaN |
| Jane Doe     | a    |     16 |
| Mary Johnson | a    |      3 |
| John Smith   | b    |      2 |
| Jane Doe     | b    |     11 |
| Mary Johnson | b    |      1 |

# 3. Tidying messy datasets

## 3.1. Column headers are values, not variable names

In [1]:
import numpy as np
import pandas as pd

The dataset in Table 4 (p.6) explores the relationship between income and religion in the US.
- It comes from a report produced by the Pew Research Center, an American think-tank that collects data on attitudes to topics ranging from religion to the internet, and produces many reports that contain datasets in this format.

```
pew = pd.read_csv('../data/pew.csv')
pew
```

In [2]:
pew = pd.read_csv('../data/pew.csv')
pew

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
0,Agnostic,27,34,60,81,76,137,122,109,84,96
1,Atheist,12,27,37,52,35,70,73,59,74,76
2,Buddhist,27,21,30,34,33,58,62,39,53,54
3,Catholic,418,617,732,670,638,1116,949,792,633,1489
4,Don’t know/refused,15,14,15,11,10,35,21,17,18,116
5,Evangelical Prot,575,869,1064,982,881,1486,949,723,414,1529
6,Hindu,1,9,7,9,11,34,47,48,54,37
7,Historically Black Prot,228,244,236,238,197,223,131,81,78,339
8,Jehovah's Witness,20,27,24,24,21,30,15,11,6,37
9,Jewish,19,19,25,25,30,95,69,87,151,162


- This dataset has three variables, `religion`, `income`, and `frequency`.
- To tidy it, we need to **melt**, or **stack** it. In other words, we need to turn columns into rows.
- Melting is parameterized by a list of columns that are already variables, or **colvar**s for short.

> [**References**] https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.melt.html

> `df.melt(id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None)`

```
pew.melt(id_vars='religion')
pew_tidy = pew.melt(id_vars='religion', var_name='income', value_name='freq')
pew_tidy.head()
```

In [3]:
pew.melt(id_vars='religion')

,religion,variable,value
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15
5,Evangelical Prot,<$10k,575
6,Hindu,<$10k,1
7,Historically Black Prot,<$10k,228
8,Jehovah's Witness,<$10k,20
9,Jewish,<$10k,19


In [4]:
pew_tidy = pew.melt(id_vars='religion', var_name='income', value_name='freq')
pew_tidy.head()

,religion,income,freq
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15


Another common use of this data format is to record regularly spaced observations over time.
- The Billboard dataset shown in Table 7 records the date a song first entered the Billboard Top 100.
- It has variables for `artist`, `track`, `date.entered`, `rank`, and `week`.
- The `rank` in each `week` after it enters the top 100 is recorded in 75 columns, `wk1` to `wk75`.
- If a song is in the Top 100 for less than 75 weeks the remaining columns are filled with missing values.

```
billboard = pd.read_csv('../data/billboard.csv')
billboard.head()
```

In [5]:
billboard = pd.read_csv('../data/billboard.csv')
billboard.head()

,year,artist,track,time,date.entered,wk1,wk2,wk3,wk4,wk5,...,wk67,wk68,wk69,wk70,wk71,wk72,wk73,wk74,wk75,wk76
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,87,82.0,72.0,77.0,87.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,91,87.0,92.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,81,70.0,68.0,67.0,66.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000,3 Doors Down,Loser,4:24,2000-10-21,76,76.0,72.0,69.0,67.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,57,34.0,25.0,17.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


- Melting yeilds Table 8.

```
billboard.melt(id_vars=['year', 'artist', 'track', 'time', 'date.entered'],
               value_name='rank',
               var_name='week')
```

In [6]:
billboard.melt(id_vars=['year', 'artist', 'track', 'time', 'date.entered'],
               value_name='rank',
               var_name='week')

,year,artist,track,time,date.entered,week,rank
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,wk1,87.0
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,wk1,91.0
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,wk1,81.0
3,2000,3 Doors Down,Loser,4:24,2000-10-21,wk1,76.0
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,wk1,57.0
5,2000,98^0,Give Me Just One Nig...,3:24,2000-08-19,wk1,51.0
6,2000,A*Teens,Dancing Queen,3:44,2000-07-08,wk1,97.0
7,2000,Aaliyah,I Don't Wanna,4:15,2000-01-29,wk1,84.0
8,2000,Aaliyah,Try Again,4:03,2000-03-18,wk1,59.0
9,2000,"Adams, Yolanda",Open My Heart,5:30,2000-08-26,wk1,76.0


- tidy data는 어떤 모습?

> [**Reference**] https://pandas.pydata.org/pandas-docs/stable/reference/series.html#api-series-str

```
billboard_tidy = billboard.melt(id_vars=['year', 'artist', 'track', 'time', 'date.entered'],
                                value_name='rank',
                                var_name='week')
billboard_tidy['week'] = billboard_tidy.week.str.extract(r'wk(\d+)')
billboard_tidy
```

In [7]:
billboard_tidy = billboard.melt(id_vars=['year', 'artist', 'track', 'time', 'date.entered'],
                                value_name='rank',
                                var_name='week')
billboard_tidy['week'] = billboard_tidy.week.str.extract(r'wk(\d+)')
billboard_tidy

,year,artist,track,time,date.entered,week,rank
0,2000,2 Pac,Baby Don't Cry (Keep...,4:22,2000-02-26,1,87.0
1,2000,2Ge+her,The Hardest Part Of ...,3:15,2000-09-02,1,91.0
2,2000,3 Doors Down,Kryptonite,3:53,2000-04-08,1,81.0
3,2000,3 Doors Down,Loser,4:24,2000-10-21,1,76.0
4,2000,504 Boyz,Wobble Wobble,3:35,2000-04-15,1,57.0
5,2000,98^0,Give Me Just One Nig...,3:24,2000-08-19,1,51.0
6,2000,A*Teens,Dancing Queen,3:44,2000-07-08,1,97.0
7,2000,Aaliyah,I Don't Wanna,4:15,2000-01-29,1,84.0
8,2000,Aaliyah,Try Again,4:03,2000-03-18,1,59.0
9,2000,"Adams, Yolanda",Open My Heart,5:30,2000-08-26,1,76.0


- 아티스트 별 `rank` 평균을 구하라.

```
(billboard_tidy
 .groupby('artist')['rank']
 .mean()
)
```

In [28]:
(billboard_tidy
 .groupby('artist')['rank']
 .mean()
)

artist
2 Pac               85.428571
2Ge+her             90.000000
3 Doors Down        37.602740
504 Boyz            56.222222
98^0                37.650000
                      ...    
Yankee Grey         83.125000
Yearwood, Trisha    84.166667
Ying Yang Twins     88.857143
Zombie Nation       99.000000
matchbox twenty     18.641026
Name: rank, Length: 228, dtype: float64

In [29]:
billboard_tidy\
 .groupby('artist')['rank']\
 .mean()

artist
2 Pac               85.428571
2Ge+her             90.000000
3 Doors Down        37.602740
504 Boyz            56.222222
98^0                37.650000
                      ...    
Yankee Grey         83.125000
Yearwood, Trisha    84.166667
Ying Yang Twins     88.857143
Zombie Nation       99.000000
matchbox twenty     18.641026
Name: rank, Length: 228, dtype: float64

- 또 다른 데이터셋, 에볼라 바이러스...

```
ebola = pd.read_csv('../data/country_timeseries.csv')
ebola
```

In [31]:
ebola = pd.read_csv('../data/country_timeseries.csv')
ebola

,Date,Day,Cases_Guinea,Cases_Liberia,Cases_SierraLeone,Cases_Nigeria,Cases_Senegal,Cases_UnitedStates,Cases_Spain,Cases_Mali,Deaths_Guinea,Deaths_Liberia,Deaths_SierraLeone,Deaths_Nigeria,Deaths_Senegal,Deaths_UnitedStates,Deaths_Spain,Deaths_Mali
0,1/5/2015,289,2776.0,NaN,10030.0,NaN,NaN,NaN,NaN,NaN,1786.0,NaN,2977.0,NaN,NaN,NaN,NaN,NaN
1,1/4/2015,288,2775.0,NaN,9780.0,NaN,NaN,NaN,NaN,NaN,1781.0,NaN,2943.0,NaN,NaN,NaN,NaN,NaN
2,1/3/2015,287,2769.0,8166.0,9722.0,NaN,NaN,NaN,NaN,NaN,1767.0,3496.0,2915.0,NaN,NaN,NaN,NaN,NaN
3,1/2/2015,286,NaN,8157.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3496.0,NaN,NaN,NaN,NaN,NaN,NaN
4,12/31/2014,284,2730.0,8115.0,9633.0,NaN,NaN,NaN,NaN,NaN,1739.0,3471.0,2827.0,NaN,NaN,NaN,NaN,NaN
5,12/28/2014,281,2706.0,8018.0,9446.0,NaN,NaN,NaN,NaN,NaN,1708.0,3423.0,2758.0,NaN,NaN,NaN,NaN,NaN
6,12/27/2014,280,2695.0,NaN,9409.0,NaN,NaN,NaN,NaN,NaN,1697.0,NaN,2732.0,NaN,NaN,NaN,NaN,NaN
7,12/24/2014,277,2630.0,7977.0,9203.0,NaN,NaN,NaN,NaN,NaN,NaN,3413.0,2655.0,NaN,NaN,NaN,NaN,NaN
8,12/21/2014,273,2597.0,NaN,9004.0,NaN,NaN,NaN,NaN,NaN,1607.0,NaN,2582.0,NaN,NaN,NaN,NaN,NaN
9,12/20/2014,272,2571.0,7862.0,8939.0,NaN,NaN,NaN,NaN,NaN,1586.0,3384.0,2556.0,NaN,NaN,NaN,NaN,NaN


- 위 데이터셋은 어떻게 멜팅?

```
ebola_long = ebola.melt(id_vars=['Date', 'Day'],
                        var_name='cd_country',
                        value_name='count')
ebola_long
```

In [32]:
ebola_long = ebola.melt(id_vars=['Date', 'Day'],
                        var_name='cd_country',
                        value_name='count')
ebola_long

,Date,Day,cd_country,count
0,1/5/2015,289,Cases_Guinea,2776.0
1,1/4/2015,288,Cases_Guinea,2775.0
2,1/3/2015,287,Cases_Guinea,2769.0
3,1/2/2015,286,Cases_Guinea,NaN
4,12/31/2014,284,Cases_Guinea,2730.0
5,12/28/2014,281,Cases_Guinea,2706.0
6,12/27/2014,280,Cases_Guinea,2695.0
7,12/24/2014,277,Cases_Guinea,2630.0
8,12/21/2014,273,Cases_Guinea,2597.0
9,12/20/2014,272,Cases_Guinea,2571.0


- `cd_country`에는 어떤 값들이 있나?

> [**Reference**] https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.unique.html

```
ebola_long.cd_country.unique()
```

In [33]:
ebola_long.cd_country.unique()

array(['Cases_Guinea', 'Cases_Liberia', 'Cases_SierraLeone',
       'Cases_Nigeria', 'Cases_Senegal', 'Cases_UnitedStates',
       'Cases_Spain', 'Cases_Mali', 'Deaths_Guinea', 'Deaths_Liberia',
       'Deaths_SierraLeone', 'Deaths_Nigeria', 'Deaths_Senegal',
       'Deaths_UnitedStates', 'Deaths_Spain', 'Deaths_Mali'], dtype=object)

- `cd_country`를 `status`와 `country` 두 컬럼으로 나누려면?

> [**Reference**] https://pandas.pydata.org/pandas-docs/stable/reference/series.html#api-series-str

```
ebola_long.cd_country.str.split('_')
ebola_split = ebola_long.cd_country.str.split('_', expand=True)
ebola_split.head()
```

In [34]:
ebola_long.cd_country.str.split('_')

0       [Cases, Guinea]
1       [Cases, Guinea]
2       [Cases, Guinea]
3       [Cases, Guinea]
4       [Cases, Guinea]
             ...       
1947     [Deaths, Mali]
1948     [Deaths, Mali]
1949     [Deaths, Mali]
1950     [Deaths, Mali]
1951     [Deaths, Mali]
Name: cd_country, Length: 1952, dtype: object

In [14]:
ebola_split = ebola_long.cd_country.str.split('_', expand=True)
ebola_split.head()

,0,1
0,Cases,Guinea
1,Cases,Guinea
2,Cases,Guinea
3,Cases,Guinea
4,Cases,Guinea


- `ebola_split`를 `ebola_long`에 넣으려면?

```
ebola_long[['status', 'country']] = ebola_split
ebola_long.head()
```

In [15]:
ebola_long[['status', 'country']] = ebola_split
ebola_long.head()

,Date,Day,cd_country,count,status,country
0,1/5/2015,289,Cases_Guinea,2776.0,Cases,Guinea
1,1/4/2015,288,Cases_Guinea,2775.0,Cases,Guinea
2,1/3/2015,287,Cases_Guinea,2769.0,Cases,Guinea
3,1/2/2015,286,Cases_Guinea,NaN,Cases,Guinea
4,12/31/2014,284,Cases_Guinea,2730.0,Cases,Guinea


- 또 다른 시도?

> [**Reference**] https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reindex.html

```
ebola_split_rev = (ebola_long
                   .cd_country
                   .str.split('_', expand=True)
                   .rename(columns={0: 'country', 1: 'status'})
                  )
ebola_split_rev.head()
```

In [16]:
ebola_split_rev = (ebola_long
                   .cd_country
                   .str.split('_', expand=True)
                   .rename(columns={0: 'status', 1: 'country'})
                   .reindex(columns=['country', 'status'])
                  )
ebola_split_rev.head()

,country,status
0,Guinea,Cases
1,Guinea,Cases
2,Guinea,Cases
3,Guinea,Cases
4,Guinea,Cases


- 왜 다를까? (**주의**)

```
ebola_long[['status', 'country']] = ebola_split_rev
ebola_long.head()

ebola_long.loc[:, ['status', 'country']] = ebola_split_rev
ebola_long.head()
```

In [17]:
ebola_long[['status', 'country']] = ebola_split_rev
ebola_long.head()

,Date,Day,cd_country,count,status,country
0,1/5/2015,289,Cases_Guinea,2776.0,Guinea,Cases
1,1/4/2015,288,Cases_Guinea,2775.0,Guinea,Cases
2,1/3/2015,287,Cases_Guinea,2769.0,Guinea,Cases
3,1/2/2015,286,Cases_Guinea,NaN,Guinea,Cases
4,12/31/2014,284,Cases_Guinea,2730.0,Guinea,Cases


In [18]:
ebola_long.loc[:, ['status', 'country']] = ebola_split_rev
ebola_long.head()

,Date,Day,cd_country,count,status,country
0,1/5/2015,289,Cases_Guinea,2776.0,Cases,Guinea
1,1/4/2015,288,Cases_Guinea,2775.0,Cases,Guinea
2,1/3/2015,287,Cases_Guinea,2769.0,Cases,Guinea
3,1/2/2015,286,Cases_Guinea,NaN,Cases,Guinea
4,12/31/2014,284,Cases_Guinea,2730.0,Cases,Guinea


## 3.3. Variables are stored in both rows and columns

The most complicated form of messy data occurs when variables are stored in both rows and columns.
- Table 11 shows daily weather data from the Global Historical Climatology Network for one weather station (MX17004) in Mexico for five months in 2010.
- It has variables in individual columns (`id`, `year`, `month`), spread across columns (`d1-d31`) and across rows (`tmin`, `tmax`) (minimum and maximum temperature).
- Months with less than 31 days have structural missing values for the last day(s) of the month.
- The element column is not a variable; it stores the names of variables.

```
weather = pd.read_csv('../data/weather.csv')
weather
```

In [36]:
weather = pd.read_csv('../data/weather.csv')
weather

,id,year,month,element,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.8,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.5,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,...,NaN,29.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,...,NaN,10.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,MX17004,2010,3,tmin,NaN,NaN,NaN,NaN,14.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,MX17004,2010,4,tmax,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,36.3,NaN,NaN,NaN,NaN
7,MX17004,2010,4,tmin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,16.7,NaN,NaN,NaN,NaN
8,MX17004,2010,5,tmax,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,33.2,NaN,NaN,NaN,NaN
9,MX17004,2010,5,tmin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,18.2,NaN,NaN,NaN,NaN


- To tidy this dataset we first melt it with colvars `id`, `year`, `month` and the column that contains variable names, `element`. This yields Table 12(a).

```
weather_long = weather.melt(id_vars=['id', 'year', 'month', 'element'],
                            var_name='day',
                            value_name='temp')
weather_long.head()
```

In [38]:
weather_long = weather.melt(id_vars=['id', 'year', 'month', 'element'],
                            var_name='day',
                            value_name='temp')
weather_long.head()

,id,year,month,element,day,temp
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN


- This dataset is mostly tidy, but we have two variables stored in rows: `tmin` and `tmax`, the type of observation.
- Fixing the issue with the type of observation requires the **cast**, or **unstack**, operation. This performs the inverse of melting by rotating the element variable back out into the columns (Table 12(b)).
- This form is tidy. There is one variable in each column, and each row represents a day's observations.

> [**Reference**] https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html

```
weather_long.pivot_table(index=['id', 'year', 'month', 'day'],
                         columns='element',
                         values='temp')
                         
(weather_long.pivot_table(index=['id', 'year', 'month', 'day'],
                          columns='element',
                          values='temp')
 .reset_index()
)
```

In [40]:
weather_long.pivot_table(index=['id', 'year', 'month', 'day'],
                         columns='element',
                         values='temp')

element                 tmax  tmin
id      year month day            
MX17004 2010 1     d30  27.8  14.5
             2     d11  29.7  13.4
                   d2   27.3  14.4
                   d23  29.9  10.7
                   d3   24.1  14.4
             3     d10  34.5  16.8
                   d16  31.1  17.6
                   d5   32.1  14.2
             4     d27  36.3  16.7
             5     d27  33.2  18.2
             6     d17  28.0  17.5
                   d29  30.1  18.0
             7     d3   28.6  17.5
                   d14  29.9  16.5
             8     d23  26.4  15.0
                   d5   29.6  15.8
                   d29  28.0  15.3
                   d13  29.8  16.5
                   d25  29.7  15.6
                   d31  25.4  15.4
                   d8   29.0  17.3
             10    d5   27.0  14.0
                   d14  29.5  13.0
                   d15  28.7  10.5
                   d28  31.2  15.0
                   d7   28.1  12.9
             11    d2   31.3  16.3
                   d5   26.3   7.9
                   d27  27.7  14.2
                   d26  28.1  12.1
                   d4   27.2  12.0
             12    d1   29.9  13.8
                   d6   27.8  10.5

In [39]:
(weather_long.pivot_table(index=['id', 'year', 'month', 'day'],
                          columns='element',
                          values='temp')
 .reset_index()
)

element,id,year,month,day,tmax,tmin
0,MX17004,2010,1,d30,27.8,14.5
1,MX17004,2010,2,d11,29.7,13.4
2,MX17004,2010,2,d2,27.3,14.4
3,MX17004,2010,2,d23,29.9,10.7
4,MX17004,2010,2,d3,24.1,14.4
5,MX17004,2010,3,d10,34.5,16.8
6,MX17004,2010,3,d16,31.1,17.6
7,MX17004,2010,3,d5,32.1,14.2
8,MX17004,2010,4,d27,36.3,16.7
9,MX17004,2010,5,d27,33.2,18.2


### **Exercise**: `year`, `month`, `day`를 합해 하나의 `date` 컬럼으로 만들어라. (e.g., `2019-01-30`)